In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_rerun_links.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 2)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 14000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:15000]:
    try:
        print(url)
        if driver_count >=20:
            driver_count = 0
            print("need to close driver")
            driver.quit()
            driver = get_driver()
            print("new driver initialize****************")
        driver.get(url)
        driver = wait_to_page_load(driver)
        try:
            agent_name = driver.find_element_by_class_name("AgentContent__name").text
        except:
            agent_name = ""
        try:
            time.sleep(0.5)
            profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
        except:
            profile_url = ""
        try:
            tag = driver.find_element_by_class_name("pill").text
        except:
            tag = ""
        try:
            role = driver.find_element_by_class_name("AgentContent__team").text
        except:
            role = ""
        try:
            location = driver.find_element_by_class_name("AgentContent__location").text
        except:
            location = ""
        contact_dict = collect_agent_contact_details(driver)
        agent_details = get_agent_details(driver)
        social_media_dict,social_media_links_str = social_media_details(driver)
        other_info = collect_other_info(driver)
        data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
        data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
        with open("kw_data-emailSelenium_14k_15k.csv",'a',newline='',encoding='utf-8') as f:
            data_df.to_csv(f, mode='a',header=f.tell()==0)
        print(url_count,url)
        url_count+=1
        driver_count+=1
        driver.delete_all_cookies()
        print("***********************************")
    except:
        pass
driver.quit()


https://kw.com/agent/UPA-6706645605956096000-0
Page load happened
14000 https://kw.com/agent/UPA-6706645605956096000-0
***********************************
https://kw.com/agent/UPA-6587385179139993601-0
Page load happened
14001 https://kw.com/agent/UPA-6587385179139993601-0
***********************************
https://kw.com/agent/UPA-6587385199420456968-2
Page load happened
14002 https://kw.com/agent/UPA-6587385199420456968-2
***********************************
https://kw.com/agent/UPA-6592948929271754758-7
Page load happened
14003 https://kw.com/agent/UPA-6592948929271754758-7
***********************************
https://kw.com/agent/UPA-6587385198436737027-2
Page load happened
14004 https://kw.com/agent/UPA-6587385198436737027-2
***********************************
https://kw.com/agent/UPA-6587385176826900483-2
Page load happened
14005 https://kw.com/agent/UPA-6587385176826900483-2
***********************************
https://kw.com/agent/UPA-6807720887246245888-8
Timeout happened no pag

Page load happened
14052 https://kw.com/agent/UPA-6592965632622051330-3
***********************************
https://kw.com/agent/UPA-6587385388196446212-1
Page load happened
14053 https://kw.com/agent/UPA-6587385388196446212-1
***********************************
https://kw.com/agent/UPA-6587385186659094529-6
Page load happened
14054 https://kw.com/agent/UPA-6587385186659094529-6
***********************************
https://kw.com/agent/UPA-6587385238212571136-4
Page load happened
14055 https://kw.com/agent/UPA-6587385238212571136-4
***********************************
https://kw.com/agent/UPA-6587385277381758980-5
Page load happened
14056 https://kw.com/agent/UPA-6587385277381758980-5
***********************************
https://kw.com/agent/UPA-6836757685771751424-2
Page load happened
14057 https://kw.com/agent/UPA-6836757685771751424-2
***********************************
https://kw.com/agent/UPA-6587385439673024515-8
Page load happened
14058 https://kw.com/agent/UPA-6587385439673024515-

Page load happened
14104 https://kw.com/agent/UPA-6587385002413461506-0
***********************************
https://kw.com/agent/UPA-6587385051731193859-1
Page load happened
14105 https://kw.com/agent/UPA-6587385051731193859-1
***********************************
https://kw.com/agent/UPA-6587385399975342084-4
Page load happened
14106 https://kw.com/agent/UPA-6587385399975342084-4
***********************************
https://kw.com/agent/UPA-6783049971317108736-2
Page load happened
14107 https://kw.com/agent/UPA-6783049971317108736-2
***********************************
https://kw.com/agent/UPA-6587385355447562244-7
Page load happened
14108 https://kw.com/agent/UPA-6587385355447562244-7
***********************************
https://kw.com/agent/UPA-6645349667750191106-6
Timeout happened no page load
14109 https://kw.com/agent/UPA-6645349667750191106-6
***********************************
https://kw.com/agent/UPA-6587385243020349440-1
Page load happened
14110 https://kw.com/agent/UPA-658738524

Timeout happened no page load
14156 https://kw.com/agent/UPA-6587385151185100801-8
***********************************
https://kw.com/agent/UPA-6587385283428216836-8
Page load happened
14157 https://kw.com/agent/UPA-6587385283428216836-8
***********************************
https://kw.com/agent/UPA-6587385288140709889-5
Timeout happened no page load
14158 https://kw.com/agent/UPA-6587385288140709889-5
***********************************
https://kw.com/agent/UPA-6587385217626656768-4
Timeout happened no page load
14159 https://kw.com/agent/UPA-6587385217626656768-4
***********************************
https://kw.com/agent/UPA-6587385306674335746-2
need to close driver
new driver initialize****************
Page load happened
14160 https://kw.com/agent/UPA-6587385306674335746-2
***********************************
https://kw.com/agent/UPA-6587385183817748482-2
Page load happened
14161 https://kw.com/agent/UPA-6587385183817748482-2
***********************************
https://kw.com/agent/UPA-

Page load happened
14208 https://kw.com/agent/UPA-6587384941832396804-6
***********************************
https://kw.com/agent/UPA-6587385224913481730-0
Page load happened
14209 https://kw.com/agent/UPA-6587385224913481730-0
***********************************
https://kw.com/agent/UPA-6787399027082792960-9
Timeout happened no page load
14210 https://kw.com/agent/UPA-6787399027082792960-9
***********************************
https://kw.com/agent/UPA-6801545341095477248-4
Page load happened
14211 https://kw.com/agent/UPA-6801545341095477248-4
***********************************
https://kw.com/agent/UPA-6593011401071976451-0
Page load happened
14212 https://kw.com/agent/UPA-6593011401071976451-0
***********************************
https://kw.com/agent/UPA-6611930419528290308-7
Page load happened
14213 https://kw.com/agent/UPA-6611930419528290308-7
***********************************
https://kw.com/agent/UPA-6587385154683097090-9
Page load happened
14214 https://kw.com/agent/UPA-658738515

...

new driver initialize****************
Page load happened
14780 https://kw.com/agent/UPA-6587385260530692098-7
***********************************
https://kw.com/agent/UPA-6587385393032450048-5
Page load happened
14781 https://kw.com/agent/UPA-6587385393032450048-5
***********************************
https://kw.com/agent/UPA-6587385195937669120-9
Page load happened
14782 https://kw.com/agent/UPA-6587385195937669120-9
***********************************
https://kw.com/agent/UPA-6587385017261834247-5
Page load happened
14783 https://kw.com/agent/UPA-6587385017261834247-5
***********************************
https://kw.com/agent/UPA-6587385008621408259-3
Page load happened
14784 https://kw.com/agent/UPA-6587385008621408259-3
***********************************
https://kw.com/agent/UPA-6663112238180642819-6
Page load happened
14785 https://kw.com/agent/UPA-6663112238180642819-6
***********************************
https://kw.com/agent/UPA-6587385325485809668-9
Page load happened
14786 https:/

Page load happened
14832 https://kw.com/agent/UPA-6736709848968126464-7
***********************************
https://kw.com/agent/UPA-6729838851674402816-4
Page load happened
14833 https://kw.com/agent/UPA-6729838851674402816-4
***********************************
https://kw.com/agent/UPA-6587385307026657281-1
Page load happened
14834 https://kw.com/agent/UPA-6587385307026657281-1
***********************************
https://kw.com/agent/UPA-6587385382036197385-4
Page load happened
14835 https://kw.com/agent/UPA-6587385382036197385-4
***********************************
https://kw.com/agent/UPA-6824467165173465088-4
Page load happened
14836 https://kw.com/agent/UPA-6824467165173465088-4
***********************************
https://kw.com/agent/UPA-6592936182206246912-5
Timeout happened no page load
14837 https://kw.com/agent/UPA-6592936182206246912-5
***********************************
https://kw.com/agent/UPA-6805486299222872064-0
Page load happened
14838 https://kw.com/agent/UPA-680548629

Page load happened
14884 https://kw.com/agent/UPA-6701994363342458880-6
***********************************
https://kw.com/agent/UPA-6587385447931723776-2
Page load happened
14885 https://kw.com/agent/UPA-6587385447931723776-2
***********************************
https://kw.com/agent/UPA-6587385259107508224-4
Page load happened
14886 https://kw.com/agent/UPA-6587385259107508224-4
***********************************
https://kw.com/agent/UPA-6587385328820879361-5
Page load happened
14887 https://kw.com/agent/UPA-6587385328820879361-5
***********************************
https://kw.com/agent/UPA-6587385003857367044-8
Page load happened
14888 https://kw.com/agent/UPA-6587385003857367044-8
***********************************
https://kw.com/agent/UPA-6587385172781142019-1
Page load happened
14889 https://kw.com/agent/UPA-6587385172781142019-1
***********************************
https://kw.com/agent/UPA-6820451139237191680-1
Page load happened
14890 https://kw.com/agent/UPA-6820451139237191680-

Timeout happened no page load
14936 https://kw.com/agent/UPA-6587385428685148162-5
***********************************
https://kw.com/agent/UPA-6587385332502794241-6
Page load happened
14937 https://kw.com/agent/UPA-6587385332502794241-6
***********************************
https://kw.com/agent/UPA-6634131748415762434-1
Page load happened
14938 https://kw.com/agent/UPA-6634131748415762434-1
***********************************
https://kw.com/agent/UPA-6587385206768816133-8
Page load happened
14939 https://kw.com/agent/UPA-6587385206768816133-8
***********************************
https://kw.com/agent/UPA-6706223621382242304-3
need to close driver
new driver initialize****************
Page load happened
14940 https://kw.com/agent/UPA-6706223621382242304-3
***********************************
https://kw.com/agent/UPA-6587385174190030850-1
Page load happened
14941 https://kw.com/agent/UPA-6587385174190030850-1
***********************************
https://kw.com/agent/UPA-6645717078462533633-6


Page load happened
14988 https://kw.com/agent/UPA-6803432953926955008-0
***********************************
https://kw.com/agent/UPA-6587384991686234116-7
Page load happened
14989 https://kw.com/agent/UPA-6587384991686234116-7
***********************************
https://kw.com/agent/UPA-6702569601971126272-5
Page load happened
14990 https://kw.com/agent/UPA-6702569601971126272-5
***********************************
https://kw.com/agent/UPA-6831669720933724160-4
Page load happened
14991 https://kw.com/agent/UPA-6831669720933724160-4
***********************************
https://kw.com/agent/UPA-6587385266299985925-1
Page load happened
14992 https://kw.com/agent/UPA-6587385266299985925-1
***********************************
https://kw.com/agent/UPA-6826580857681453056-2
Page load happened
14993 https://kw.com/agent/UPA-6826580857681453056-2
***********************************
https://kw.com/agent/UPA-6626587624099356672-3
Page load happened
14994 https://kw.com/agent/UPA-6626587624099356672-

In [4]:
driver.quit()